# Cell width

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Imports

In [2]:
%load_ext autoreload
%autoreload 2
import sys 

# sys.path.append('..')
from omegaconf import OmegaConf
from pprint import pprint
from dacite import from_dict
from dacite import Config as DaciteConfig
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import pandas as pd
import numpy as np

from xlstm.xlstm_block_stack import xLSTMBlockStack, xLSTMBlockStackConfig

from xlstm_moex.data.download import get_historical_data

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Init config

In [2]:
# xlstm_cfg = f""" 
# mlstm_block:
#   mlstm:
#     conv1d_kernel_size: 4
#     qkv_proj_blocksize: 1
#     num_heads: 1
#     proj_factor: 1
# slstm_block:
#   slstm:
#     backend: {'cuda' if torch.cuda.is_available() else 'vanilla'} #! only vanilla here works
#     num_heads: 1
#     conv1d_kernel_size: 0
#     bias_init: powerlaw_blockdependent
#   feedforward:
#     proj_factor: 1.2
#     act_fn: gelu
# context_length: 50
# num_blocks: 2
# embedding_dim: 1 # same as `in_features` in Pytorch LSTM
# slstm_at: [1] #[1] # for [] it also works, so if no sLSTM is in the stack
# """

xlstm_cfg = f""" 
mlstm_block:
  mlstm:
    conv1d_kernel_size: 4
    qkv_proj_blocksize: 4
    num_heads: 4
slstm_block:
  slstm:
    backend: {'cuda' if torch.cuda.is_available() else 'vanilla'} #! only vanilla here works
    num_heads: 4
    conv1d_kernel_size: 4
    bias_init: powerlaw_blockdependent
  feedforward:
    proj_factor: 1.3
    act_fn: gelu
context_length: 256
num_blocks: 7
embedding_dim: 128
add_post_blocks_norm: False
slstm_at: [1] #[1] # for [] it also works, so if no sLSTM is in the stack
"""

# Init XLSTM model

In [3]:
cfg = OmegaConf.create(xlstm_cfg)
cfg = from_dict(data_class=xLSTMBlockStackConfig, data=OmegaConf.to_container(cfg), config=DaciteConfig(strict=True))
xlstm_stack = xLSTMBlockStack(cfg)

{'verbose': True, 'with_cuda': True, 'extra_ldflags': ['-L/home/nick/anaconda3/envs/nxai_xlstm/lib', '-lcublas'], 'extra_cflags': ['-DSLSTM_HIDDEN_SIZE=128', '-DSLSTM_BATCH_SIZE=8', '-DSLSTM_NUM_HEADS=4', '-DSLSTM_NUM_STATES=4', '-DSLSTM_DTYPE_B=float', '-DSLSTM_DTYPE_R=__nv_bfloat16', '-DSLSTM_DTYPE_W=__nv_bfloat16', '-DSLSTM_DTYPE_G=__nv_bfloat16', '-DSLSTM_DTYPE_S=__nv_bfloat16', '-DSLSTM_DTYPE_A=float', '-DSLSTM_NUM_GATES=4', '-DSLSTM_SIMPLE_AGG=true', '-DSLSTM_GRADIENT_RECURRENT_CLIPVAL_VALID=false', '-DSLSTM_GRADIENT_RECURRENT_CLIPVAL=0.0', '-DSLSTM_FORWARD_CLIPVAL_VALID=false', '-DSLSTM_FORWARD_CLIPVAL=0.0', '-U__CUDA_NO_HALF_OPERATORS__', '-U__CUDA_NO_HALF_CONVERSIONS__', '-U__CUDA_NO_BFLOAT16_OPERATORS__', '-U__CUDA_NO_BFLOAT16_CONVERSIONS__', '-U__CUDA_NO_BFLOAT162_OPERATORS__', '-U__CUDA_NO_BFLOAT162_CONVERSIONS__'], 'extra_cuda_cflags': ['-Xptxas="-v"', '-gencode', 'arch=compute_80,code=compute_80', '-res-usage', '--use_fast_math', '-O3', '-Xptxas -O3', '--extra-device-vect

Using /home/nick/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/nick/.cache/torch_extensions/py311_cu121/slstm_HS128BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0/build.ninja...
Building extension module slstm_HS128BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module slstm_HS128BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0...


# Inspect config

In [5]:
pprint(cfg.embedding_dim)

128


# Inspect layers

In [91]:
xlstm_stack

xLSTMBlockStack(
  (blocks): ModuleList(
    (0): mLSTMBlock(
      (xlstm_norm): LayerNorm()
      (xlstm): mLSTMLayer(
        (proj_up): Linear(in_features=128, out_features=512, bias=False)
        (q_proj): LinearHeadwiseExpand(in_features=256, num_heads=64, expand_factor_up=1, bias=False, trainable_weight=True, trainable_bias=True, )
        (k_proj): LinearHeadwiseExpand(in_features=256, num_heads=64, expand_factor_up=1, bias=False, trainable_weight=True, trainable_bias=True, )
        (v_proj): LinearHeadwiseExpand(in_features=256, num_heads=64, expand_factor_up=1, bias=False, trainable_weight=True, trainable_bias=True, )
        (conv1d): CausalConv1d(
          (conv): Conv1d(256, 256, kernel_size=(4,), stride=(1,), padding=(3,), groups=256)
        )
        (conv_act_fn): SiLU()
        (mlstm_cell): mLSTMCell(
          (igate): Linear(in_features=768, out_features=4, bias=True)
          (fgate): Linear(in_features=768, out_features=4, bias=True)
          (outnorm): Mult

# Generate synthetic examlpe

In [122]:
x = torch.randn(2, 256, 128).to(device=device)

In [123]:
x.shape

torch.Size([2, 256, 128])

In [97]:
x[:, 1, :].view(2,1,128).shape

torch.Size([2, 1, 128])

# Check model's output

In [98]:
xlstm_stack = xlstm_stack.to(device=device)

In [84]:
for i in range(2):
    y, states_dict = xlstm_stack.step(x[:, i, :].view(2,1,128))

In [87]:
states_dict['block_6']

{'mlstm_state': (tensor([[[[ 5.4874e-06, -1.2764e-06, -4.2204e-06,  ..., -1.7712e-05,
             -2.4697e-06, -7.9438e-06],
            [ 4.2978e-05, -9.9968e-06, -3.3054e-05,  ..., -1.3872e-04,
             -1.9342e-05, -6.2216e-05],
            [-2.4128e-05,  5.6123e-06,  1.8557e-05,  ...,  7.7881e-05,
              1.0859e-05,  3.4929e-05],
            ...,
            [-1.0465e-05,  2.4343e-06,  8.0490e-06,  ...,  3.3780e-05,
              4.7100e-06,  1.5150e-05],
            [-3.4788e-05,  8.0919e-06,  2.6756e-05,  ...,  1.1229e-04,
              1.5657e-05,  5.0361e-05],
            [-1.0198e-05,  2.3721e-06,  7.8432e-06,  ...,  3.2916e-05,
              4.5896e-06,  1.4763e-05]],
  
           [[ 1.1881e-04, -4.0994e-04,  2.5100e-04,  ...,  1.2454e-06,
             -2.5543e-04,  3.8301e-04],
            [ 1.7321e-04, -5.9762e-04,  3.6592e-04,  ...,  1.8156e-06,
             -3.7237e-04,  5.5836e-04],
            [ 7.2518e-05, -2.5021e-04,  1.5320e-04,  ...,  7.6015e-07,
     

In [81]:
y.shape

torch.Size([2, 1, 128])

In [82]:
y

tensor([[[-0.4978,  1.5445,  1.2798, -1.4677, -2.1897,  0.7250,  0.6384,
          -0.3543, -0.6553, -0.5294,  0.9895,  0.8925,  0.0369,  0.3600,
           1.0671, -0.8967,  0.7769,  1.7208, -0.9602, -2.3994, -0.3734,
          -0.0866,  1.4061,  0.0087, -0.7891, -0.0173, -1.9538, -0.3937,
          -0.5488, -0.5184,  1.9376, -0.7643, -0.7506,  1.1501, -0.1561,
           0.4513, -1.2406, -0.9217,  0.3434,  0.6700, -0.2392,  0.1344,
          -0.0117, -0.9852, -1.2051,  0.3428, -0.1593, -1.8795, -0.0729,
           1.1581,  0.8664,  0.4295, -0.0295,  0.2280,  1.7564,  0.3873,
          -0.2153,  1.3113,  0.0999,  0.7173, -1.0597, -2.2170,  1.2809,
          -1.7322,  0.5820,  1.3519,  0.9275, -0.4028,  0.1005,  0.5075,
          -0.2481,  0.8091,  0.7399, -0.1949,  0.4818, -0.6697,  0.0183,
           0.0428, -0.2187, -1.7713,  1.3467, -1.7573,  0.5467,  1.1787,
           1.0028, -0.7336,  1.9220,  0.9596, -0.5030, -1.3327,  1.2682,
           0.0318,  0.2944, -0.4946, -1.2384, -0.16

In [83]:
x[:, 1, :].view(2,1,128)

tensor([[[-8.8171e-01,  7.5984e-01,  1.3308e+00, -8.0911e-01, -7.1669e-01,
           1.2324e-01,  6.4953e-01,  1.9917e-01, -2.8007e-02,  2.0094e-01,
           8.0642e-01,  1.5618e+00, -3.5212e-01,  3.7325e-02,  9.7363e-01,
          -6.5723e-01, -6.3216e-01, -2.7189e-01, -6.8714e-01, -2.0393e+00,
          -3.4308e-02,  9.7309e-01, -2.3616e-01, -3.9586e-01, -6.4092e-02,
           9.2531e-01, -2.7790e+00, -6.8198e-01, -1.6832e-02, -2.0715e-01,
           2.5983e+00, -7.3946e-01, -1.0449e+00,  7.0546e-01,  1.0123e+00,
           7.0937e-01, -5.6492e-01, -1.9830e+00, -2.7441e-01,  8.7571e-01,
           1.7398e-02,  9.0983e-01,  1.3253e-01,  4.8852e-02, -1.4840e+00,
          -2.9523e-02,  1.3925e+00,  1.0114e-01,  3.2748e-01,  1.0978e+00,
          -8.5351e-01, -3.0628e-01, -4.6161e-01, -3.8436e-01,  1.0335e+00,
           6.7830e-01, -3.3133e-01,  6.6896e-01, -4.6368e-01,  7.4131e-01,
          -1.6848e+00, -1.2495e+00,  1.8919e+00, -6.0618e-01,  2.2224e+00,
           9.6635e-01,  4

In [124]:
y = xlstm_stack(x)

In [125]:
x[:, -1, :]

tensor([[-0.8839,  0.6531, -0.8390, -0.1645,  1.6704,  0.5127, -0.0529,  0.2271,
         -2.5904,  1.3150,  0.3769, -0.1745,  0.2888,  0.9407, -0.1206,  0.6557,
          0.1330,  0.1118,  0.4273,  0.5373, -0.3927, -0.0368, -0.3819,  1.1103,
          0.2225,  0.5210, -0.3059, -0.2255, -0.9224, -0.4568, -2.2560,  1.0137,
         -0.1084, -0.6037,  0.5965, -0.5597, -0.5377, -0.5736,  0.8941, -2.7295,
          1.3067,  0.7518,  0.9739,  0.5398, -0.4596,  0.3005, -0.3543, -0.9072,
          0.3558,  0.1623,  0.5433, -0.9490,  0.1389, -1.0022, -1.1491, -0.3044,
         -0.1395, -0.2475, -0.1755,  0.2795,  1.3190,  0.0260, -0.0259, -0.6402,
         -0.3443, -1.1712,  0.2273, -0.3507,  1.4633, -0.2927,  1.0462,  1.0782,
          0.0841, -0.4016,  0.4079,  0.1693,  0.1225,  0.6326,  0.0118, -1.4028,
         -1.1295,  0.6233,  0.1547,  0.0442, -0.7434, -0.5714, -0.8025,  0.6202,
          0.8696,  1.3488,  0.3080,  0.0314, -0.9079, -0.9560, -0.4306,  0.1162,
         -0.6938,  0.3198, -

In [126]:
y[:, -1, :]

tensor([[-8.8777e-01,  7.0733e-01, -1.1354e+00,  6.8297e-02,  1.2867e+00,
          8.6632e-01,  8.6892e-01,  9.2484e-01, -1.2396e+00,  1.7039e+00,
         -1.0790e+00, -1.0183e+00, -6.6037e-01,  3.6460e-01,  9.4218e-01,
         -9.8400e-02, -1.5814e+00,  4.7198e-01,  1.7564e+00,  1.6153e+00,
          2.3493e-01,  2.9874e-01, -6.3176e-01,  7.8221e-01,  2.1035e+00,
          3.4908e-01, -2.4726e+00,  9.4857e-01, -6.0690e-01,  6.2530e-01,
         -4.8214e+00,  1.9170e+00, -3.0467e-01, -5.0227e-01,  8.7043e-01,
         -4.5736e-01,  4.8147e-03, -5.5708e-01,  2.0257e-01, -2.0661e+00,
          9.1403e-01,  1.8090e+00,  2.6168e-01,  1.3294e+00, -2.0494e+00,
         -1.7052e+00, -5.7928e-01, -7.9424e-01,  9.7212e-01,  2.6963e-01,
          2.0599e+00,  3.2527e-02,  1.6426e+00, -1.7454e+00,  1.4678e+00,
         -6.0697e-01,  5.2550e-01,  4.7229e-01, -5.6340e-01, -1.4185e+00,
          1.2872e+00, -1.8675e+00, -5.2358e-01, -6.4815e-01, -3.6819e-01,
         -1.1029e+00,  3.5903e-01, -6.

In [100]:
y.shape

torch.Size([2, 256, 128])

In [54]:
pprint(y)

tensor([[[ 0.8577, -1.1099, -0.3923,  ...,  0.0940,  0.0937,  0.7333],
         [ 0.8142,  0.0359,  0.3861,  ..., -0.1010, -0.7600, -0.0066],
         [ 2.4055, -1.4971,  0.7610,  ..., -0.5647,  1.4988, -1.2182],
         ...,
         [-1.9958,  0.2881,  0.2207,  ...,  0.1564, -1.6645, -0.3780],
         [-0.4549,  0.3158, -0.3409,  ...,  0.4510, -0.3832,  1.3363],
         [-0.8845, -0.6907,  0.5125,  ...,  1.1808, -1.4142,  1.1701]],

        [[ 0.8740, -2.0196,  0.3545,  ..., -0.7747, -0.1946, -1.2063],
         [-0.3995,  0.4758,  0.2324,  ...,  1.6014, -0.7852,  0.3335],
         [ 0.2124, -0.1924, -0.0830,  ...,  0.5713,  1.6000,  1.5129],
         ...,
         [-0.1742,  1.3780, -0.5989,  ...,  0.0386, -0.5372,  1.5216],
         [ 0.4534,  0.9240, -0.4873,  ...,  0.6691, -0.8149,  1.2498],
         [-0.2121, -0.4231, -1.2146,  ...,  0.4834,  1.7986, -0.7707]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)


In [103]:
pprint(y[:, -1, :].view(1, -1, 128))

tensor([[[ 1.1480, -1.0595,  2.0452, -0.0863, -0.1370,  2.0175, -0.4361,
          -0.1611, -1.5872,  1.1023, -2.6816,  0.6252,  1.0867, -1.2995,
          -0.2884,  1.1804,  1.3891, -1.3155,  0.1274, -3.0621, -4.0834,
           1.0835,  0.5426,  0.4090,  1.8845,  0.7048, -2.5606, -0.6336,
          -0.5072, -0.1934, -0.1107,  4.0092, -0.6715, -0.6509, -1.7275,
           0.2459,  0.1660,  1.4571, -1.2948, -2.2396, -1.9346, -0.5495,
           1.1377, -0.1067,  0.4261, -0.1087, -0.5682,  5.4285,  0.8706,
           1.8102, -1.4242,  0.7603, -0.2171, -1.3306, -0.8624,  1.6319,
          -0.9979, -0.4409,  0.2527, -0.4009, -2.9918, -2.7180,  0.6599,
          -0.0263, -0.8976,  1.0564,  0.7353, -0.1279, -0.2253, -0.8580,
           1.5953, -1.0426,  2.8536, -2.9724, -2.6748,  0.6674,  0.2355,
           0.4488,  3.3318, -1.6286,  0.3944, -1.5674,  0.0701,  0.0965,
           2.1487, -0.0439, -0.0311, -1.9621,  2.6495,  1.4265,  1.3333,
           0.7164, -3.4484, -1.8838, -0.6050,  2.54

In [92]:
t = ()
for i in range(5):
    t += (i,)

In [93]:
t[0]

0

In [128]:
from torch import nn

rnn = nn.LSTM(10, 20, 2, batch_first=True)
inpt = torch.randn(5, 3, 10)
h0 = torch.randn(2, 5, 20)
c0 = torch.randn(2, 5, 20)
output, (hn, cn) = rnn(inpt, (h0, c0))

In [129]:
output.shape

torch.Size([5, 3, 20])

In [130]:
hn.shape

torch.Size([2, 5, 20])

In [121]:
hn[0]

tensor([[-0.2291,  0.0706, -0.2873, -0.0193,  0.0982,  0.1942,  0.0228, -0.1276,
          0.0790,  0.0730, -0.1216,  0.0562,  0.1600,  0.1633, -0.0206, -0.0394,
         -0.0367, -0.0733,  0.1518, -0.1802],
        [-0.1929,  0.1946,  0.0548, -0.1062,  0.0587,  0.0891, -0.0085, -0.0888,
          0.0230,  0.2114,  0.1167,  0.1953,  0.0377, -0.0480, -0.0815,  0.1854,
          0.0207, -0.0434, -0.0672, -0.1421],
        [-0.2902,  0.2361, -0.2112, -0.0509, -0.0608,  0.1652, -0.0098, -0.1355,
          0.1445,  0.1078,  0.0399,  0.1191,  0.0123, -0.0428,  0.0851, -0.0684,
         -0.2345, -0.0469,  0.2690,  0.1379],
        [-0.2975,  0.0864,  0.1136, -0.1358,  0.0094,  0.1324,  0.0084, -0.1017,
         -0.1106,  0.1565, -0.1292,  0.1546, -0.0853, -0.1034,  0.0358,  0.1535,
         -0.0611, -0.0737, -0.1640, -0.1349],
        [-0.0851,  0.1141,  0.2472, -0.2493, -0.1072,  0.0791, -0.1482, -0.1104,
          0.1480,  0.1121,  0.0041,  0.1225,  0.1576,  0.0570, -0.0797, -0.0238,
      

In [119]:
output[:,-1,:]

tensor([[-0.2291,  0.0706, -0.2873, -0.0193,  0.0982,  0.1942,  0.0228, -0.1276,
          0.0790,  0.0730, -0.1216,  0.0562,  0.1600,  0.1633, -0.0206, -0.0394,
         -0.0367, -0.0733,  0.1518, -0.1802],
        [-0.1929,  0.1946,  0.0548, -0.1062,  0.0587,  0.0891, -0.0085, -0.0888,
          0.0230,  0.2114,  0.1167,  0.1953,  0.0377, -0.0480, -0.0815,  0.1854,
          0.0207, -0.0434, -0.0672, -0.1421],
        [-0.2902,  0.2361, -0.2112, -0.0509, -0.0608,  0.1652, -0.0098, -0.1355,
          0.1445,  0.1078,  0.0399,  0.1191,  0.0123, -0.0428,  0.0851, -0.0684,
         -0.2345, -0.0469,  0.2690,  0.1379],
        [-0.2975,  0.0864,  0.1136, -0.1358,  0.0094,  0.1324,  0.0084, -0.1017,
         -0.1106,  0.1565, -0.1292,  0.1546, -0.0853, -0.1034,  0.0358,  0.1535,
         -0.0611, -0.0737, -0.1640, -0.1349],
        [-0.0851,  0.1141,  0.2472, -0.2493, -0.1072,  0.0791, -0.1482, -0.1104,
          0.1480,  0.1121,  0.0041,  0.1225,  0.1576,  0.0570, -0.0797, -0.0238,
      

# Model prototype

## Network

In [3]:
class xLSTMtime(nn.Module):
    def __init__(self, input_size, output_size, xlstm_blockstack_config):
        super(xLSTMtime, self).__init__()
        self.fc_in = nn.Linear(in_features=input_size, out_features=xlstm_blockstack_config.embedding_dim)
        self.xlstm = xLSTMBlockStack(xlstm_blockstack_config)
        self.fc_out = nn.Linear(in_features=xlstm_blockstack_config.embedding_dim, out_features=output_size)

    def forward(self, x):
        x = self.fc_in(x)
        x = self.xlstm(x)
        x = self.fc_out(x)
        return x

## Test config

In [4]:
test_cfg_str = f""" 
mlstm_block:
  mlstm:
    conv1d_kernel_size: 4
    qkv_proj_blocksize: 4
    num_heads: 4
slstm_block:
  slstm:
    backend: {'cuda' if torch.cuda.is_available() else 'vanilla'} #! only vanilla here works
    num_heads: 4
    conv1d_kernel_size: 4
    bias_init: powerlaw_blockdependent
  feedforward:
    proj_factor: 1.3
    act_fn: gelu
context_length: 50
num_blocks: 3
embedding_dim: 16
add_post_blocks_norm: False
slstm_at: [1] #[1] # for [] it also works, so if no sLSTM is in the stack
"""

test_cfg = OmegaConf.create(test_cfg_str)
test_cfg = from_dict(data_class=xLSTMBlockStackConfig, data=OmegaConf.to_container(test_cfg), config=DaciteConfig(strict=True))

## Init model

In [5]:
xlstmtime = xLSTMtime(
    input_size=1,
    output_size=1,
    xlstm_blockstack_config=test_cfg
)

{'verbose': True, 'with_cuda': True, 'extra_ldflags': ['-L/home/nick/anaconda3/envs/nxai_xlstm/lib', '-lcublas'], 'extra_cflags': ['-DSLSTM_HIDDEN_SIZE=16', '-DSLSTM_BATCH_SIZE=8', '-DSLSTM_NUM_HEADS=4', '-DSLSTM_NUM_STATES=4', '-DSLSTM_DTYPE_B=float', '-DSLSTM_DTYPE_R=__nv_bfloat16', '-DSLSTM_DTYPE_W=__nv_bfloat16', '-DSLSTM_DTYPE_G=__nv_bfloat16', '-DSLSTM_DTYPE_S=__nv_bfloat16', '-DSLSTM_DTYPE_A=float', '-DSLSTM_NUM_GATES=4', '-DSLSTM_SIMPLE_AGG=true', '-DSLSTM_GRADIENT_RECURRENT_CLIPVAL_VALID=false', '-DSLSTM_GRADIENT_RECURRENT_CLIPVAL=0.0', '-DSLSTM_FORWARD_CLIPVAL_VALID=false', '-DSLSTM_FORWARD_CLIPVAL=0.0', '-U__CUDA_NO_HALF_OPERATORS__', '-U__CUDA_NO_HALF_CONVERSIONS__', '-U__CUDA_NO_BFLOAT16_OPERATORS__', '-U__CUDA_NO_BFLOAT16_CONVERSIONS__', '-U__CUDA_NO_BFLOAT162_OPERATORS__', '-U__CUDA_NO_BFLOAT162_CONVERSIONS__'], 'extra_cuda_cflags': ['-Xptxas="-v"', '-gencode', 'arch=compute_80,code=compute_80', '-res-usage', '--use_fast_math', '-O3', '-Xptxas -O3', '--extra-device-vecto

Using /home/nick/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/nick/.cache/torch_extensions/py311_cu121/slstm_HS16BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0/build.ninja...
Building extension module slstm_HS16BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module slstm_HS16BS8NH4NS4DBfDRbDWbDGbDSbDAfNG4SA1GRCV0GRC0d0FCV0FC0d0...


In [6]:
xlstmtime = xlstmtime.to(device=device)

## Test input

In [7]:
test_input = torch.randn(2, 50, 1).to(device=device)

In [8]:
test_output = xlstmtime(test_input)

In [9]:
test_output[:,-1,:].shape

torch.Size([2, 1])

In [10]:
test_input[:,-1,:]

tensor([[ 0.3219],
        [-1.0757]], device='cuda:0')

## Test dataloader

In [11]:
# await get_historical_data(
#     output_filename='yndx_20170901_20200901.csv',
#     stock='YNDX',
#     start_date='2017-09-01',
#     end_date='2020-09-01'
# )

In [12]:
test_data = (
    pd
    .read_csv('yndx_20170901_20200901.csv')
    .sort_values(by=['TRADEDATE'], ascending=[True])
    ['OPEN']
    .to_list()
)

In [13]:
len(test_data)

756

In [14]:
X_examples = []
y_examples = []
for i in range(len(test_data) - 50 - 1):
    X_examples.append(test_data[i:i+50])
    y_examples.append(test_data[i+50])
X_examples = np.array(X_examples)
y_examples = np.array(y_examples)

In [15]:
X_examples

array([[1746.5, 1817. , 1831.5, ..., 1944. , 1933.5, 1890. ],
       [1817. , 1831.5, 1861.5, ..., 1933.5, 1890. , 1856. ],
       [1831.5, 1861.5, 1849. , ..., 1890. , 1856. , 1874.5],
       ...,
       [2984.2, 3024.8, 3058.6, ..., 4800. , 4920. , 5060.4],
       [3024.8, 3058.6, 3108. , ..., 4920. , 5060.4, 4993.6],
       [3058.6, 3108. , 3170.2, ..., 5060.4, 4993.6, 4870.2]])

In [16]:
y_examples

array([1856. , 1874.5, 1890. , 1889. , 1926.5, 1965. , 2001. , 2030. ,
       1980.5, 1981.5, 2033. , 2019. , 2002.5, 1949.5, 1935. , 1932. ,
       1900. , 1899.5, 1920. , 1949.5, 1960. , 1996. , 1935. , 1935. ,
       1915. , 1895. , 1892. , 1898. , 1885.5, 1865. , 1832.5, 1852. ,
       1845. , 1867.5, 1880. , 1919. , 1943. , 1966. , 1957.5, 1963. ,
       1963.5, 1955. , 1958. , 1964. , 1993. , 2050. , 2112. , 2095.5,
       2131.5, 2159.5, 2138.5, 2135. , 2191.5, 2176.5, 2160.5, 2183. ,
       2190.5, 2145.5, 2073. , 2106. , 2137. , 2090.5, 2101.5, 2124. ,
       2155. , 2250. , 2341.5, 2371. , 2420. , 2424.5, 2469. , 2450. ,
       2363. , 2381.5, 2329.5, 2309. , 2313.5, 2377.5, 2399. , 2474. ,
       2465. , 2449. , 2425. , 2442.5, 2441. , 2435. , 2427.5, 2465. ,
       2487.5, 2390. , 2377. , 2360.5, 2339.5, 2279. , 2288.5, 2290. ,
       2250. , 2255. , 2264.5, 2313. , 2310. , 2090.5, 2168.5, 2211. ,
       2139.5, 2051. , 2148.5, 2080. , 2160. , 2111. , 2095. , 2142. ,
      

In [17]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        super().__init__()
        self.X = torch.from_numpy(X.astype('float32')).to(device=device).reshape(-1, 50 , 1)
        self.y = torch.from_numpy(y.astype('float32')).to(device=device)
        print(self.X.shape)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index].unsqueeze(0)

In [18]:
dataset = TimeSeriesDataset(X=X_examples, y=y_examples)

torch.Size([705, 50, 1])


In [19]:
dataloader = DataLoader(
    dataset=dataset,
    batch_size=32,
    shuffle=True
)

In [20]:
for X_batch, y_batch in dataloader:
    print(X_batch.shape)
    print(y_batch.shape)
    break

torch.Size([32, 50, 1])
torch.Size([32, 1])


## Test train loop

In [21]:
criterion = nn.MSELoss()
optimizer = optim.Adam(xlstmtime.parameters(), lr=0.01)

In [22]:
num_epochs = 500
for epoch in range(num_epochs):
    xlstmtime.train()
    for batch_idx, (X_b, y_b) in enumerate(dataloader):
        optimizer.zero_grad()
        y_b_pred = xlstmtime(X_b)
        y_b_pred = y_b_pred[:,-1,:] # select state for last element in sequence
        loss = criterion(y_b_pred, y_b)
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

Epoch [1/500], Batch [5/23], Loss: 525909.6875
Epoch [1/500], Batch [10/23], Loss: 112678.0938
Epoch [1/500], Batch [15/23], Loss: 264791.5000
Epoch [1/500], Batch [20/23], Loss: 37600.4492
Epoch [2/500], Batch [5/23], Loss: 53250.7383
Epoch [2/500], Batch [10/23], Loss: 22515.0000
Epoch [2/500], Batch [15/23], Loss: 4963.0923
Epoch [2/500], Batch [20/23], Loss: 13830.1445
Epoch [3/500], Batch [5/23], Loss: 2805.5898
Epoch [3/500], Batch [10/23], Loss: 4656.0732
Epoch [3/500], Batch [15/23], Loss: 2581.5762
Epoch [3/500], Batch [20/23], Loss: 2295.8210
Epoch [4/500], Batch [5/23], Loss: 2676.9995
Epoch [4/500], Batch [10/23], Loss: 3180.3267
Epoch [4/500], Batch [15/23], Loss: 2387.1011
Epoch [4/500], Batch [20/23], Loss: 3496.9402
Epoch [5/500], Batch [5/23], Loss: 2981.3496
Epoch [5/500], Batch [10/23], Loss: 3260.6243
Epoch [5/500], Batch [15/23], Loss: 5995.5913
Epoch [5/500], Batch [20/23], Loss: 1658.5823
Epoch [6/500], Batch [5/23], Loss: 1544.5593
Epoch [6/500], Batch [10/23], 

Epoch [45/500], Batch [15/23], Loss: 2292.1318
Epoch [45/500], Batch [20/23], Loss: 1531.8344
Epoch [46/500], Batch [5/23], Loss: 1444.2363
Epoch [46/500], Batch [10/23], Loss: 5465.8994
Epoch [46/500], Batch [15/23], Loss: 1911.7239
Epoch [46/500], Batch [20/23], Loss: 3630.7888
Epoch [47/500], Batch [5/23], Loss: 1296.8921
Epoch [47/500], Batch [10/23], Loss: 2605.0479
Epoch [47/500], Batch [15/23], Loss: 2716.8328
Epoch [47/500], Batch [20/23], Loss: 3290.8320
Epoch [48/500], Batch [5/23], Loss: 720.4559
Epoch [48/500], Batch [10/23], Loss: 1607.8583
Epoch [48/500], Batch [15/23], Loss: 6445.1406
Epoch [48/500], Batch [20/23], Loss: 3760.4595
Epoch [49/500], Batch [5/23], Loss: 959.4189
Epoch [49/500], Batch [10/23], Loss: 3865.3804
Epoch [49/500], Batch [15/23], Loss: 3159.1924
Epoch [49/500], Batch [20/23], Loss: 5330.9746
Epoch [50/500], Batch [5/23], Loss: 2361.0549
Epoch [50/500], Batch [10/23], Loss: 2003.9792
Epoch [50/500], Batch [15/23], Loss: 4069.8569
Epoch [50/500], Batc

Epoch [89/500], Batch [20/23], Loss: 2235.5925
Epoch [90/500], Batch [5/23], Loss: 9633.1592
Epoch [90/500], Batch [10/23], Loss: 3238.7998
Epoch [90/500], Batch [15/23], Loss: 2196.7900
Epoch [90/500], Batch [20/23], Loss: 2257.2231
Epoch [91/500], Batch [5/23], Loss: 4002.8926
Epoch [91/500], Batch [10/23], Loss: 7191.5493
Epoch [91/500], Batch [15/23], Loss: 1887.4539
Epoch [91/500], Batch [20/23], Loss: 4793.0137
Epoch [92/500], Batch [5/23], Loss: 1930.2026
Epoch [92/500], Batch [10/23], Loss: 2330.6177
Epoch [92/500], Batch [15/23], Loss: 1416.1548
Epoch [92/500], Batch [20/23], Loss: 4440.9116
Epoch [93/500], Batch [5/23], Loss: 1993.2611
Epoch [93/500], Batch [10/23], Loss: 6068.4810
Epoch [93/500], Batch [15/23], Loss: 2099.0801
Epoch [93/500], Batch [20/23], Loss: 2420.1094
Epoch [94/500], Batch [5/23], Loss: 2386.1340
Epoch [94/500], Batch [10/23], Loss: 3872.7864
Epoch [94/500], Batch [15/23], Loss: 1969.6316
Epoch [94/500], Batch [20/23], Loss: 1073.7394
Epoch [95/500], Ba

Epoch [133/500], Batch [15/23], Loss: 2809.9922
Epoch [133/500], Batch [20/23], Loss: 1274.3938
Epoch [134/500], Batch [5/23], Loss: 2418.7656
Epoch [134/500], Batch [10/23], Loss: 8268.4316
Epoch [134/500], Batch [15/23], Loss: 3158.8682
Epoch [134/500], Batch [20/23], Loss: 2657.6250
Epoch [135/500], Batch [5/23], Loss: 2282.4863
Epoch [135/500], Batch [10/23], Loss: 1810.8452
Epoch [135/500], Batch [15/23], Loss: 6167.7910
Epoch [135/500], Batch [20/23], Loss: 1043.1510
Epoch [136/500], Batch [5/23], Loss: 1532.1732
Epoch [136/500], Batch [10/23], Loss: 3359.7112
Epoch [136/500], Batch [15/23], Loss: 3670.9307
Epoch [136/500], Batch [20/23], Loss: 2296.3652
Epoch [137/500], Batch [5/23], Loss: 1700.4159
Epoch [137/500], Batch [10/23], Loss: 1800.1833
Epoch [137/500], Batch [15/23], Loss: 1840.0750
Epoch [137/500], Batch [20/23], Loss: 3156.7200
Epoch [138/500], Batch [5/23], Loss: 1671.3950
Epoch [138/500], Batch [10/23], Loss: 1654.3689
Epoch [138/500], Batch [15/23], Loss: 1836.57

Epoch [177/500], Batch [10/23], Loss: 3744.3584
Epoch [177/500], Batch [15/23], Loss: 3738.5120
Epoch [177/500], Batch [20/23], Loss: 2316.9241
Epoch [178/500], Batch [5/23], Loss: 5685.3340
Epoch [178/500], Batch [10/23], Loss: 3404.7529
Epoch [178/500], Batch [15/23], Loss: 2409.9043
Epoch [178/500], Batch [20/23], Loss: 3911.3584
Epoch [179/500], Batch [5/23], Loss: 2680.6260
Epoch [179/500], Batch [10/23], Loss: 4317.6182
Epoch [179/500], Batch [15/23], Loss: 2197.7588
Epoch [179/500], Batch [20/23], Loss: 2640.0276
Epoch [180/500], Batch [5/23], Loss: 2218.1387
Epoch [180/500], Batch [10/23], Loss: 1439.3127
Epoch [180/500], Batch [15/23], Loss: 3154.6792
Epoch [180/500], Batch [20/23], Loss: 5669.5771
Epoch [181/500], Batch [5/23], Loss: 4120.5430
Epoch [181/500], Batch [10/23], Loss: 2223.2754
Epoch [181/500], Batch [15/23], Loss: 9943.7480
Epoch [181/500], Batch [20/23], Loss: 2785.6458
Epoch [182/500], Batch [5/23], Loss: 3440.4211
Epoch [182/500], Batch [10/23], Loss: 1990.71

...

Epoch [306/500], Batch [15/23], Loss: 1795.7606
Epoch [306/500], Batch [20/23], Loss: 5166.0991
Epoch [307/500], Batch [5/23], Loss: 9193.7188
Epoch [307/500], Batch [10/23], Loss: 7497.7134
Epoch [307/500], Batch [15/23], Loss: 11189.7598
Epoch [307/500], Batch [20/23], Loss: 4841.9966
Epoch [308/500], Batch [5/23], Loss: 4668.4424
Epoch [308/500], Batch [10/23], Loss: 5450.5859
Epoch [308/500], Batch [15/23], Loss: 6169.8413
Epoch [308/500], Batch [20/23], Loss: 3211.2461
Epoch [309/500], Batch [5/23], Loss: 2783.0869
Epoch [309/500], Batch [10/23], Loss: 1523.4912
Epoch [309/500], Batch [15/23], Loss: 2427.3833
Epoch [309/500], Batch [20/23], Loss: 3387.6182
Epoch [310/500], Batch [5/23], Loss: 984.6142
Epoch [310/500], Batch [10/23], Loss: 3772.5947
Epoch [310/500], Batch [15/23], Loss: 3470.1394
Epoch [310/500], Batch [20/23], Loss: 2161.1030
Epoch [311/500], Batch [5/23], Loss: 1958.7573
Epoch [311/500], Batch [10/23], Loss: 1848.1641
Epoch [311/500], Batch [15/23], Loss: 2461.27

Epoch [350/500], Batch [5/23], Loss: 1618.5869
Epoch [350/500], Batch [10/23], Loss: 3255.1875
Epoch [350/500], Batch [15/23], Loss: 2956.9478
Epoch [350/500], Batch [20/23], Loss: 3489.0696
Epoch [351/500], Batch [5/23], Loss: 2535.0320
Epoch [351/500], Batch [10/23], Loss: 3868.1589
Epoch [351/500], Batch [15/23], Loss: 3805.8960
Epoch [351/500], Batch [20/23], Loss: 2053.1428
Epoch [352/500], Batch [5/23], Loss: 1467.7554
Epoch [352/500], Batch [10/23], Loss: 2567.2310
Epoch [352/500], Batch [15/23], Loss: 6889.7754
Epoch [352/500], Batch [20/23], Loss: 1885.2804
Epoch [353/500], Batch [5/23], Loss: 3245.6572
Epoch [353/500], Batch [10/23], Loss: 2770.2593
Epoch [353/500], Batch [15/23], Loss: 5118.0068
Epoch [353/500], Batch [20/23], Loss: 5344.7427
Epoch [354/500], Batch [5/23], Loss: 994.8878
Epoch [354/500], Batch [10/23], Loss: 1681.4827
Epoch [354/500], Batch [15/23], Loss: 3489.3276
Epoch [354/500], Batch [20/23], Loss: 1785.1262
Epoch [355/500], Batch [5/23], Loss: 2152.8396

Epoch [393/500], Batch [20/23], Loss: 1126.9038
Epoch [394/500], Batch [5/23], Loss: 1751.1653
Epoch [394/500], Batch [10/23], Loss: 3247.1372
Epoch [394/500], Batch [15/23], Loss: 1958.2056
Epoch [394/500], Batch [20/23], Loss: 2509.3960
Epoch [395/500], Batch [5/23], Loss: 3636.7349
Epoch [395/500], Batch [10/23], Loss: 2908.4800
Epoch [395/500], Batch [15/23], Loss: 3281.1445
Epoch [395/500], Batch [20/23], Loss: 1574.4088
Epoch [396/500], Batch [5/23], Loss: 3720.0547
Epoch [396/500], Batch [10/23], Loss: 1065.7046
Epoch [396/500], Batch [15/23], Loss: 3391.1589
Epoch [396/500], Batch [20/23], Loss: 929.8355
Epoch [397/500], Batch [5/23], Loss: 7276.8140
Epoch [397/500], Batch [10/23], Loss: 1005.9125
Epoch [397/500], Batch [15/23], Loss: 2828.3096
Epoch [397/500], Batch [20/23], Loss: 3758.0815
Epoch [398/500], Batch [5/23], Loss: 3409.8093
Epoch [398/500], Batch [10/23], Loss: 1164.1807
Epoch [398/500], Batch [15/23], Loss: 1767.0109
Epoch [398/500], Batch [20/23], Loss: 1682.122

Epoch [437/500], Batch [15/23], Loss: 4444.3716
Epoch [437/500], Batch [20/23], Loss: 2427.2632
Epoch [438/500], Batch [5/23], Loss: 5339.3828
Epoch [438/500], Batch [10/23], Loss: 2676.1245
Epoch [438/500], Batch [15/23], Loss: 6955.1045
Epoch [438/500], Batch [20/23], Loss: 2614.6875
Epoch [439/500], Batch [5/23], Loss: 6698.7837
Epoch [439/500], Batch [10/23], Loss: 1737.5444
Epoch [439/500], Batch [15/23], Loss: 3477.0522
Epoch [439/500], Batch [20/23], Loss: 2666.3369
Epoch [440/500], Batch [5/23], Loss: 4140.0542
Epoch [440/500], Batch [10/23], Loss: 2536.8467
Epoch [440/500], Batch [15/23], Loss: 1548.5020
Epoch [440/500], Batch [20/23], Loss: 4513.1748
Epoch [441/500], Batch [5/23], Loss: 2319.7012
Epoch [441/500], Batch [10/23], Loss: 1777.7269
Epoch [441/500], Batch [15/23], Loss: 3438.6829
Epoch [441/500], Batch [20/23], Loss: 2204.3091
Epoch [442/500], Batch [5/23], Loss: 4044.9490
Epoch [442/500], Batch [10/23], Loss: 3398.1772
Epoch [442/500], Batch [15/23], Loss: 2792.68

Epoch [481/500], Batch [5/23], Loss: 2409.5063
Epoch [481/500], Batch [10/23], Loss: 3514.9783
Epoch [481/500], Batch [15/23], Loss: 2578.9302
Epoch [481/500], Batch [20/23], Loss: 2044.7725
Epoch [482/500], Batch [5/23], Loss: 4086.8262
Epoch [482/500], Batch [10/23], Loss: 2238.1445
Epoch [482/500], Batch [15/23], Loss: 2847.3320
Epoch [482/500], Batch [20/23], Loss: 2858.7561
Epoch [483/500], Batch [5/23], Loss: 1442.5381
Epoch [483/500], Batch [10/23], Loss: 4166.3931
Epoch [483/500], Batch [15/23], Loss: 2489.1211
Epoch [483/500], Batch [20/23], Loss: 1910.7345
Epoch [484/500], Batch [5/23], Loss: 1215.8702
Epoch [484/500], Batch [10/23], Loss: 3751.3975
Epoch [484/500], Batch [15/23], Loss: 3206.8997
Epoch [484/500], Batch [20/23], Loss: 1209.3870
Epoch [485/500], Batch [5/23], Loss: 1383.6061
Epoch [485/500], Batch [10/23], Loss: 3595.7844
Epoch [485/500], Batch [15/23], Loss: 1087.9666
Epoch [485/500], Batch [20/23], Loss: 5088.2612
Epoch [486/500], Batch [5/23], Loss: 1329.598

## Examine outputs

In [33]:
X_after_train = torch.from_numpy(X_examples[228:229,:].astype('float32')).to(device=device).reshape(-1, 50 , 1)
y_after_train = y_examples[228:229]

In [34]:
output_after_train = xlstmtime(X_after_train)

In [35]:
output_after_train[:,-1,:]

tensor([[2171.7285]], device='cuda:0', grad_fn=<SliceBackward0>)

In [36]:
y_after_train

array([2151.])

In [38]:
X_examples[228:229,:]

array([[2384.5, 2350. , 2240.5, 2257. , 2266. , 2260.5, 2260. , 2241. ,
        2255. , 2142. , 2134. , 2140. , 2165. , 2142. , 2105.5, 2123.5,
        2108. , 2125. , 2163.5, 2150. , 2118. , 2142. , 2158. , 2159. ,
        2170. , 2132. , 2159. , 2183.5, 2170. , 2122.5, 2080. , 2100.5,
        2123.5, 2097. , 2070.5, 2145. , 2192. , 2117. , 2140. , 2152.5,
        2178. , 2202.5, 2196.5, 2213. , 2194. , 2195. , 2168. , 2136. ,
        2143.5, 2185.5]])

In [40]:
X_examples[229:231,:]

array([[2350. , 2240.5, 2257. , 2266. , 2260.5, 2260. , 2241. , 2255. ,
        2142. , 2134. , 2140. , 2165. , 2142. , 2105.5, 2123.5, 2108. ,
        2125. , 2163.5, 2150. , 2118. , 2142. , 2158. , 2159. , 2170. ,
        2132. , 2159. , 2183.5, 2170. , 2122.5, 2080. , 2100.5, 2123.5,
        2097. , 2070.5, 2145. , 2192. , 2117. , 2140. , 2152.5, 2178. ,
        2202.5, 2196.5, 2213. , 2194. , 2195. , 2168. , 2136. , 2143.5,
        2185.5, 2151. ],
       [2240.5, 2257. , 2266. , 2260.5, 2260. , 2241. , 2255. , 2142. ,
        2134. , 2140. , 2165. , 2142. , 2105.5, 2123.5, 2108. , 2125. ,
        2163.5, 2150. , 2118. , 2142. , 2158. , 2159. , 2170. , 2132. ,
        2159. , 2183.5, 2170. , 2122.5, 2080. , 2100.5, 2123.5, 2097. ,
        2070.5, 2145. , 2192. , 2117. , 2140. , 2152.5, 2178. , 2202.5,
        2196.5, 2213. , 2194. , 2195. , 2168. , 2136. , 2143.5, 2185.5,
        2151. , 2143. ]])